In [386]:
import http.client
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

conn = http.client.HTTPSConnection("www.congreso.es")

payload = "_diputadomodule_idLegislatura=14&_diputadomodule_genero=2&_diputadomodule_grupo=all&_diputadomodule_tipo=0&_diputadomodule_nombre=&_diputadomodule_apellidos=&_diputadomodule_formacion=all&_diputadomodule_filtroProvincias=%5B%5D&_diputadomodule_nombreCircunscripcion="

headers = {
    'accept': "application/json, text/javascript, */*; q=0.01",
    'accept-language': "en-US,en;q=0.6",
    'content-type': "application/x-www-form-urlencoded; charset=UTF-8",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36",
    'x-requested-with': "XMLHttpRequest"
    }

conn.request("POST", "/busqueda-de-diputados?p_p_id=diputadomodule&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_resource_id=searchDiputados&p_p_cacheability=cacheLevelPage", payload, headers)

res = conn.getresponse()

data = res.read()

info = data.decode("utf-8")

responseObject = json.loads(info)['data']


In [429]:
# 4m 7s

codParlamentario = [x['codParlamentario'] for x in responseObject] # list of all Parlamentarios ID

for idx,codParlamentario in enumerate(codParlamentario):
    
    row = {}

    url = f'https://www.congreso.es/busqueda-de-diputados?p_p_id=diputadomodule&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_diputadomodule_mostrarFicha=true&codParlamentario={codParlamentario}&idLegislatura=XIV&mostrarAgenda=false'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    responseObject[idx]['birthday'] = soup.find_all('p')[1].text.strip()[10:39]
    
    if soup.find_all('p')[2].text.strip()[-1] == 's':
        responseObject[idx]['legislaturas'] = soup.find_all('p')[2].text.strip()[:-23]
    else:
        responseObject[idx]['legislaturas'] = soup.find_all('p')[2].text.strip()


In [472]:
data_profile = pd.DataFrame(responseObject)

columns_to_drop = ['fchBaja', 'genero','fchAlta', 'idLegislatura']

data_profile.drop(columns_to_drop,axis=1, inplace=True)

data_profile['birthday'] = data_profile['birthday'].apply(lambda x: pd.to_datetime(x))

pd.set_option('display.max_rows', 10)

data_profile

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,apellidos,formacion,apellidosNombre,grupo,idCircunscripcion,nombreCircunscripcion,nombre,codParlamentario,birthday,legislaturas
0,Aizpurua Arzallus,EH Bildu,"Aizpurua Arzallus, Mertxe",Grupo Parlamentario Euskal Herria Bildu,20,Gipuzkoa,Mertxe,69,1960-01-08,"Diputada de la XIII, y XIV"
1,Alonso Suárez,PsdeG-PSOE,"Alonso Suárez, María Olga",Grupo Parlamentario Socialista,36,Pontevedra,María Olga,72,1962-07-12,"Diputada de la XIII, y XIV"
2,Álvarez de Toledo Peralta-Ramos,PP,"Álvarez de Toledo Peralta-Ramos, Cayetana",Grupo Parlamentario Popular en el Congreso,8,Barcelona,Cayetana,308,1974-10-15,"Diputada de la IX, X, XIII, y XIV"
3,Álvarez Fanjul,PP,"Álvarez Fanjul, Beatriz",Grupo Parlamentario Popular en el Congreso,48,Bizkaia,Beatriz,101,1991-08-23,Diputada de la XIV Legislatura
4,Andrés Añón,PSC-PSOE,"Andrés Añón, Carmen",Grupo Parlamentario Socialista,8,Barcelona,Carmen,204,1964-10-19,"Diputada de la XIII, y XIV"
...,...,...,...,...,...,...,...,...,...,...
143,Velasco Morillo,PP,"Velasco Morillo, Elvira",Grupo Parlamentario Popular en el Congreso,49,Zamora,Elvira,10,1968-03-05,"Diputada de la VIII, XIII, y XIV"
144,Vidal Sáez,ECP-GUAYEM EL CANVI,"Vidal Sáez, Aina",Grupo Parlamentario Confederal de Unidas Podem...,8,Barcelona,Aina,266,1985-03-22,"Diputada de la XI, XII, XIII, y XIV"
145,Vilches Ruiz,PSOE,"Vilches Ruiz, María Luisa",Grupo Parlamentario Socialista,2,Albacete,María Luisa,332,1982-01-16,"Diputada de la XIII, y XIV"
146,Villagrasa Quero,PSOE,"Villagrasa Quero, Noemí",Grupo Parlamentario Socialista,50,Zaragoza,Noemí,49,1980-09-14,"Diputada de la XIII, y XIV"


In [465]:
pd.set_option('display.max_rows', None)

def years(year):
    try:
        (pd.Timestamp.now() - year) / np.timedelta64(1, 'Y')
    except:
        pd.to_datetime(year)

data_profile[data_profile.codParlamentario!=7].birthday.apply(lambda x: (pd.Timestamp.now() - pd.to_datetime(x)) / np.timedelta64(1, 'Y')).mean()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


49.97540593127791

In [466]:
data_profile[data_profile.codParlamentario!=7].birthday.apply(lambda x: (pd.Timestamp.now() - pd.to_datetime(x)) / np.timedelta64(1, 'Y')).median()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


49.90978963868136

In [469]:
data_profile[data_profile.codParlamentario!=7].birthday.apply(lambda x: (pd.Timestamp.now() - pd.to_datetime(x)) / np.timedelta64(1, 'Y')).min()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname CEST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


26.42950107240763

In [473]:
data_profile[data_profile.codParlamentario!=7].birthday.min()

Timestamp('1949-05-15 00:00:00')